In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/DontGetKicked/example_entry.csv
/kaggle/input/DontGetKicked/training.zip
/kaggle/input/DontGetKicked/Carvana_Data_Dictionary.txt
/kaggle/input/DontGetKicked/test.zip
/kaggle/input/DontGetKicked/training.csv
/kaggle/input/DontGetKicked/test.csv


# Introductory Summary

1. Rank 16th / 570 (Late submission)

2. Feature Engineering of DateTime Variables

3. Replacing NaN with Zero

4. Label Encoding for Categorical Features

In [2]:
train = pd.read_csv('/kaggle/input/DontGetKicked/training.csv')
test = pd.read_csv('/kaggle/input/DontGetKicked/test.zip')

In [3]:
train

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,1,0,12/7/2009,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,...,11597.0,12409.0,NaN,NaN,21973,33619,FL,7100.0,0,1113
1,2,0,12/7/2009,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,...,11374.0,12791.0,NaN,NaN,19638,33619,FL,7600.0,0,1053
2,3,0,12/7/2009,ADESA,2005,4,DODGE,STRATUS V6,SXT,4D SEDAN SXT FFV,...,7146.0,8702.0,NaN,NaN,19638,33619,FL,4900.0,0,1389
3,4,0,12/7/2009,ADESA,2004,5,DODGE,NEON,SXT,4D SEDAN,...,4375.0,5518.0,NaN,NaN,19638,33619,FL,4100.0,0,630
4,5,0,12/7/2009,ADESA,2005,4,FORD,FOCUS,ZX3,2D COUPE ZX3,...,6739.0,7911.0,NaN,NaN,19638,33619,FL,4000.0,0,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72978,73010,1,12/2/2009,ADESA,2001,8,MERCURY,SABLE,GS,4D SEDAN GS,...,4836.0,5937.0,NaN,NaN,18111,30212,GA,4200.0,0,993
72979,73011,0,12/2/2009,ADESA,2007,2,CHEVROLET,MALIBU 4C,LS,4D SEDAN LS,...,10151.0,11652.0,NaN,NaN,18881,30212,GA,6200.0,0,1038
72980,73012,0,12/2/2009,ADESA,2005,4,JEEP,GRAND CHEROKEE 2WD V,Lar,4D WAGON LAREDO,...,11831.0,14402.0,NaN,NaN,18111,30212,GA,8200.0,0,1893
72981,73013,0,12/2/2009,ADESA,2006,3,CHEVROLET,IMPALA,LS,4D SEDAN LS,...,10099.0,11228.0,NaN,NaN,18881,30212,GA,7000.0,0,1974


In [4]:
test

,RefId,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,73015,12/2/2009,ADESA,2005,4,PONTIAC,GRAND PRIX,Bas,4D SEDAN,SILVER,...,8557.0,9752.0,NaN,NaN,18881,30212,GA,6500.0,0,2152
1,73016,12/2/2009,ADESA,2005,4,CHEVROLET,MALIBU V6,LS,4D SEDAN LS,SILVER,...,7562.0,9296.0,NaN,NaN,18111,30212,GA,6300.0,0,1118
2,73017,12/2/2009,ADESA,2006,3,DODGE,DURANGO 2WD V8,Adv,4D SUV 4.7L ADVENTURER,SILVER,...,15340.0,16512.0,NaN,NaN,18111,30212,GA,9700.0,0,1215
3,73018,12/2/2009,ADESA,2002,7,SATURN,L SERIES,L20,4D SEDAN L200,GOLD,...,5725.0,6398.0,NaN,NaN,18881,30212,GA,4150.0,0,1933
4,73019,12/2/2009,ADESA,2007,2,HYUNDAI,ACCENT,GS,2D COUPE GS,BLUE,...,5914.0,7350.0,NaN,NaN,18111,30212,GA,4100.0,0,920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48702,121742,11/17/2010,MANHEIM,2005,5,FORD,FIVE HUNDRED,SEL,4D SEDAN SEL,BLACK,...,9764.0,11395.0,NaN,NaN,20928,33411,FL,7955.0,0,1633
48703,121743,11/17/2010,MANHEIM,2007,3,TOYOTA,COROLLA,CE,4D SEDAN CE,GREEN,...,10283.0,11565.0,NaN,NaN,20928,33411,FL,7035.0,0,594
48704,121744,11/17/2010,MANHEIM,2006,4,KIA,SPECTRA,EX,4D SEDAN EX,BLACK,...,7871.0,9490.0,NO,GREEN,20928,33411,FL,6335.0,0,594
48705,121745,11/17/2010,MANHEIM,2005,5,MAZDA,MAZDA3,s,4D SEDAN GT,SILVER,...,8576.0,9937.0,NO,GREEN,20928,33411,FL,8055.0,0,1038


## Preprocessing
- 날짜데이터 feature extraction
- 결측치 처리
- label encoding

In [5]:
alldata = pd.concat([train,test])

alldata['PurchDate'] = pd.to_datetime(alldata['PurchDate'])
alldata['Year'] = alldata['PurchDate'].dt.year
alldata['Month'] = alldata['PurchDate'].dt.month
alldata['Day'] = alldata['PurchDate'].dt.day

alldata2 = alldata.drop(columns = ['PurchDate','IsBadBuy','RefId'])

In [6]:
alldata2 = alldata2.fillna(0)

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_cols = alldata2.columns[alldata2.dtypes == object]
for i in cat_cols:
    alldata2[i] = le.fit_transform(list(alldata2[i]))

In [8]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

## Modeling

In [9]:
from catboost import CatBoostClassifier
cbc = CatBoostClassifier(verbose = 100)
cbc.fit(train2,train['IsBadBuy'])
result = cbc.predict_proba(test2)

Learning rate set to 0.064347
0:	learn: 0.6367842	total: 84.7ms	remaining: 1m 24s
100:	learn: 0.2929173	total: 1.91s	remaining: 17s
200:	learn: 0.2837950	total: 3.65s	remaining: 14.5s
300:	learn: 0.2754287	total: 5.41s	remaining: 12.6s
400:	learn: 0.2683758	total: 7.16s	remaining: 10.7s
500:	learn: 0.2620957	total: 8.87s	remaining: 8.84s
600:	learn: 0.2563358	total: 10.5s	remaining: 7s
700:	learn: 0.2509843	total: 12.3s	remaining: 5.23s
800:	learn: 0.2460486	total: 14s	remaining: 3.48s
900:	learn: 0.2411165	total: 15.7s	remaining: 1.73s
999:	learn: 0.2364138	total: 17.4s	remaining: 0us


In [10]:
sub = pd.read_csv('/kaggle/input/DontGetKicked/example_entry.csv')
sub['IsBadBuy'] = result[:,1]
sub.to_csv('sub.csv', index = 0)

In [11]:
sub = pd.read_csv('/kaggle/input/DontGetKicked/example_entry.csv')
sub['IsBadBuy'] = result[:,1]
sub.to_csv('sub.csv', index = 0)

## Feature Importance

In [12]:
pd.Series(cbc.feature_importances_,index = train2.columns).sort_values(ascending = False)

WheelTypeID                          8.639382
VNZIP1                               6.073894
VehBCost                             5.689453
WheelType                            5.665378
Month                                5.564246
BYRNO                                5.442246
VehOdo                               4.556845
MMRCurrentAuctionAveragePrice        3.632494
SubModel                             3.232207
MMRCurrentRetailCleanPrice           3.132773
Auction                              3.061604
Model                                3.036745
VehYear                              3.023424
MMRCurrentRetailAveragePrice         3.016504
VehicleAge                           2.983788
Day                                  2.948150
WarrantyCost                         2.907155
MMRCurrentAuctionCleanPrice          2.895196
MMRAcquisitionRetailAveragePrice     2.732071
VNST                                 2.687489
MMRAcquisitionAuctionAveragePrice    2.531796
Trim                              

In [13]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121690 entries, 0 to 48706
Data columns (total 37 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   RefId                              121690 non-null  int64         
 1   IsBadBuy                           72983 non-null   float64       
 2   PurchDate                          121690 non-null  datetime64[ns]
 3   Auction                            121690 non-null  object        
 4   VehYear                            121690 non-null  int64         
 5   VehicleAge                         121690 non-null  int64         
 6   Make                               121690 non-null  object        
 7   Model                              121690 non-null  object        
 8   Trim                               117780 non-null  object        
 9   SubModel                           121677 non-null  object        
 10  Color                